## Prepare

In [ ]:

import pandas as pd
import numpy as np

     


import regex as re
import nltk
def tokensize(text):
    return re.findall(r'[\w-]*\p{L}[\w-]*',text)

nltk.download("all")

     

un = pd.read_csv("/content/drive/MyDrive/UN/un-general-debates.csv")

     

stopwords = set(nltk.corpus.stopwords.words('english'))

def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stopwords]
     

pipeline = [str.lower,tokensize,remove_stop]

def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens
     

un['tokens'] = un['text'].apply(prepare, pipeline=pipeline)
un['num_tokens'] = un['tokens'].map(len) 

# N-gram Analysis

동일 단어를 활용해도 단어의 순서 혹은 합성어에 따라서 의미는 달라진다.

>해결: 빈도 분석의 대상을 단일 단어에서 두 세 단어의 짧은 시퀀스로 확장하는 것이 도움된다.

합성어와 연어라는 두 가지 유형의 단어 시퀀스를 찾는데 합성어는 특정한 의미를 지닌 두 개 이상의 단어가 조합된 말이다.

그러므로, 두 개의 토큰을 하나의 의미로 간주해야 하고 이와 대조적으로 연어란 자주 사용되는 단어 조합이다.

그러므로, 텍스트 처리에는 주로 두 단어의 조합, 세 단어의 조합으로 작업한다.



N-gram의 크기가 1이면 단일 단어이며 유니그램으로 부르고, n<=3을 고수하는 이유 n값이 커질수록 서로 다른 N-그램의 수는 기하급수적으로 증가하며 빈도는 기하급수적으로 감소하기 때문이다. 

In [2]:
"""
바이어그램에는 대부분의 전치사 및 한정사같은 불용어가 포함되므로 불용어없이 바이어그램을 작성하는 것이 좋다.
불용어를 제거한 다음에 바이그램을 빌드하면 원본텍스트에 존재하지 않는 바이그램이 생성되므로 수정된 ngrams함수를 사용해서 
모든 토큰에 대한 바이그램을 생성한 뒤 불용어를 포함하지 않는 토큰만 유지한다.
"""
def ngrams(tokens, n=2, sep=' '):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

text = "the visible mainfestation of the global climate change"
tokens = tokensize(text)
print("|".join(ngrams(tokens,2)))

the visible|visible mainfestation|mainfestation of|of the|the global|global climate|climate change


In [3]:
"""
ngrams 함수는 모든 바이그램을 포함한 열을 데이터프레임에 추가
앞서 단어 계수에 사용한 count_words를 적용해 상위 5개 바이그램 결정
"""
def ngrams(tokens, n=2, sep=' ', stopwords=set()):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])
            if len([t for t in ngram if t in stopwords])==0]
print("Bigrams:", "|".join(ngrams(tokens,2,stopwords=stopwords)))
print("Trigrams:", "|".join(ngrams(tokens,3,stopwords=stopwords)))


Bigrams: visible mainfestation|global climate|climate change
Trigrams: global climate change


In [4]:
un['bigrams'] = un['text'].apply(prepare, pipeline=[str.lower, tokensize]) \
                          .apply(ngrams,n=2, stopwords=stopwords)

In [5]:
"""
자세하게 추가 처리 및 분석을 위해선 Counter 클래스를 팬더스 데이터프레임으로 변환하는 것이 훨씬 더 편하다
토큰은 데이터프레임의 인덱스가 되고, 빈도값은 freq라는 열에 저장되고, 가장 자주 사용되는 단어가 행의 첫 부분에 표시되도록 정렬된다.
"""

"""
판다스의 데이터프레임을 첫 번째 매개변수로 사용하고, 토큰 혹은 텍스트를 포함한 열 이름을 두 번째 매개변수로 사용한다.

데이터프레임의 토큰 열에 연설을 토큰화한 토큰이 저장
"""

from collections import Counter
counter = Counter()
def count_words(df, column="tokens", preprocess=None, min_freq=2):

    #토큰들을 처리하고 counter를 업데이트한다.
    def update(doc):
        tokens =doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

        #counter를 생성하고, 모든 데이터에 대해 update를 실행한다.
    counter = Counter()
    df[column].map(update)

        #counter를 DataFrame화
    freq_df = pd.DataFrame.from_dict(counter, orient="index",columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq') #빈도수가 최솟값(0) 이상인 것들을 뽑아라
    freq_df.index.name = 'token'
    return freq_df.sort_values("freq", ascending=False)

In [6]:
count_words(un, 'bigrams').head()

,freq
token,
united nations,103236
international community,27786
general assembly,27096
security council,20961
human rights,19856


문장 경계 무시로 인해서 생성되는 이상한 바이그램은 실제론 중요하지 않다.

그러나, 이를 방지하려면 문장 경계를 식별해야하는데 토큰화보다 더 복잡하다.

TF-IDF 기반의 유니그램 분석을 확장해 바이그램을 포함하고, 바이그램의 idf값을 추가하여 모든 연설에 대해 가중 바이그램 빈도를 계산한 결과 데이터프레임에서 워드 클라우드를 생성한다.

In [7]:
"""
count_words와 거의 동일하지만, 각 토큰이 문서당 한 번만 계산 (counter.update(set(tokens)))된다.

IDF값이 용어의 총 개수를 계산한 후에 계산되므로 매개변수 mid_df는 사용빈도가 낮은 롱테일에 대한 필터 역할을 한다.

아래의 함수는 데이터프레임을 반환한다.
"""

def compute_idf(df, column='tokens', preprocess=None, min_df=2):

    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(set(tokens))

    #토큰 개수를 얻는다.
    counter = Counter()
    df[column].map(update)

    #데이터프레임 생성 후 idf를 계산한다.

    idf_df = pd.DataFrame.from_dict(counter, orient='index',columns=['df'])
    idf_df = idf_df.query('df >= @min_df')
    idf_df['idf'] = np.log(len(df)/idf_df['df']) + 0.1
    idf_df.index.name = 'token'
    return idf_df

In [8]:
idf_df = compute_idf(un)


<ipython-input-7-6fc370bd1dce>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  idf_df['idf'] = np.log(len(df)/idf_df['df']) + 0.1


In [10]:
"""
count_words에 의해 생성된 빈도값을 포함하는 판다스의 시리즈도 지원하도록
함수를 둘러싼 작은 래퍼를 준비하였고, WordCloud클래스에는 결과를 미세 조정할 수 있는 옵션이 존재하여,
이 중 일부를 사용해서 결과를 조정한다.
"""

"""
아래 함수는 단어를 필터링하는 편의 매개 변수가 두 개가 있는데 skip_n은 목록의 상위 n개 단어를 건너뛴다.

유엔 말뭉치에는 분명 united, Nations, Interational같은 단어가 빈도 목록의 앞에 나와서
나열된 단어를 시각화하는 것이 더 흥미롭다.

두 번째 필터는 불용어를 추가한 목록으로 자주 사용되지만 시각화를 위해 흥미롭지 않은 특정 단어를 필터링 하는 것이 도움이 된다.
"""


from wordcloud import WordCloud
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400,
                   background_color="black", colormap="Paired",
                   max_font_size=150, max_words=max_words)
    
    #데이터프레임을 사전형으로 변경한다.
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    #빈도 counter에서 불용어를 필터링한다.
    if stopwords is not None:
        counter = {token:freq for (token,freq) in counter.items()
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)

    plt.title(title)
    
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")

In [ ]:
idf_df = pd.concat([idf_df, compute_idf(un,'bigrams',min_df=10)])

freq_df = count_words(un[un['year'] ==2015], 'bigrams')
freq_df['tfidf'] = freq_df['freq'] + idf_df['idf']
wordcloud(freq_df['tfidf'], title='all bigrams', max_words=50)

In [ ]:
where = freq_df.index.str.contains("climate")
wordcloud(freq_df[where]['freq'], title='"climate" bigrams', max_words=50)

불용어를 포함하지 않은 모든 N-그램을 생성하고 가중치를 부여하는데 이 결과는 상당히 좋다.

<br>

드물게 나타나는 롱테일 바이그램은 신경 쓰지 않는다.

NLTK의 언어 감지는 계산 비용이 많이 들지만 더 정교한 알고리즘을 사용해 연어를 식별한다.